In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from enum import Enum
import numpy as np

df = pd.read_excel('survey-results.xlsx')
TOTAL_RESPONDENTS = len(df)

print(df.columns)

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Q7', 'Q6', 'Q8', 'Q9', 'Q10',
       'Q1_1', 'Q1_2', 'Q1_3', 'Q1_4', 'Q1_5', 'Q1_6', 'Q1_7', 'Q1_8', 'Q1_9',
       'Q1_10', 'Q3_1', 'Q3_2', 'Q3_3', 'Q3_4', 'Q3_5', 'Q3_6', 'Q5_1', 'Q5_2',
       'Q5_3', 'Q5_4', 'Q5_5', 'Q5_6', 'Q5_7', 'Q5_8', 'Q5_9', 'Q5_10',
       'Q5_11', 'Q5_12', 'Q5_13', 'Q5_14', 'Q5_15'],
      dtype='object')


In [9]:
ages_count = df['Q8'].value_counts().sort_index(ascending=True)
not_answered_df = pd.Series([TOTAL_RESPONDENTS - ages_count.sum()], index=['Not Answered'])
ages_count = pd.concat([ages_count, not_answered_df])

gender_count = df['Q9'].value_counts().sort_index(ascending=True)
not_answered_df = pd.Series([TOTAL_RESPONDENTS - gender_count.sum()], index=['Not Answered'])
gender_count = pd.concat([gender_count, not_answered_df])

injury_count = df['Q10'].value_counts().sort_index(ascending=True)
not_answered_df = pd.Series([TOTAL_RESPONDENTS - injury_count.sum()], index=['Not Answered'])
injury_count = pd.concat([injury_count, not_answered_df])

print(ages_count)
print(gender_count)
print(injury_count)

16 - 24                  2
Please enter your age    1
Not Answered             0
dtype: int64
Female                      1
Male                        1
Please enter your gender    1
Not Answered                0
dtype: int64
Have you ever been injured whilst cycling?    1
Yes - major injury                            1
Yes - minor injury                            1
Not Answered                                  0
dtype: int64


In [37]:
ratings = ["Strongly disagree", "Somewhat disagree", "Neutral", "Somewhat agree", "Strongly agree"]
factors = ["PD", "MD", "TD", "P", "E", "F"]
comparisons = [["P", "MD"],["E", "P"],["PD", "P"],["PD", "F"],["TD", "E"],["P", "TD"],["E", "PD"],["F", "MD"],["TD", "F"],["MD", "PD"],["F", "E"],["P", "F"],["TD", "MD"],["PD", "TD"],["MD", "E"]]
sus_scores = {}
sus_totals = []
nasa_scores = {}


for p in range(1, TOTAL_RESPONDENTS):
    sus_scores[p] = []
    nasa_scores[p] = {}
    final_score = 0
            
    #print("\nGET SUS SCORES")
    for q in range(1, 11):
        que = 'Q1_' + str(q)
        rating = df[que][p]
        score = ratings.index(rating)
        if q % 2 == 0: 
            score = 5 - (score+1)
        sus_scores[p].append(score)
    sus_totals.append(sum(sus_scores[p])*2.5)
    
    #print("\nGET NASA-TLX RATINGS")
    for q in range(1, 7):
        que = 'Q3_' + str(q)
        rating = (df[que][p]) * 5
        nasa_scores[p][factors[q-1]] = {}
        nasa_scores[p][factors[q-1]]["count"] = 0
        nasa_scores[p][factors[q-1]]["rating"] = rating
    
    #print("\nGET NASA-TLX WEIGHTINGS")
    for q in range(1, 16):
        que = 'Q5_' + str(q)
        n = int(df[que][p])
        factor = comparisons[q-1][n-1]
        nasa_scores[p][factor]["count"] += 1
            
    for f in factors:
        weight = nasa_scores[p][f]["count"] / 15
        rating = nasa_scores[p][f]["rating"]
        score = rating * weight
        nasa_scores[p][f]["score"] = score
        final_score += score
        
    nasa_scores[p]["final_score"] = final_score

print(f"Sus totals: {sus_totals} with an average sus score of {np.mean(sus_totals):.2f}")
print(nasa_scores)

Sus totals: [45.0, 52.5] with an average sus score of 48.75
{1: {'PD': {'count': 4, 'rating': 90, 'score': 24.0}, 'MD': {'count': 2, 'rating': 45, 'score': 6.0}, 'TD': {'count': 2, 'rating': 60, 'score': 8.0}, 'P': {'count': 2, 'rating': 25, 'score': 3.3333333333333335}, 'E': {'count': 3, 'rating': 45, 'score': 9.0}, 'F': {'count': 2, 'rating': 100, 'score': 13.333333333333334}, 'final_score': 63.66666666666667}, 2: {'PD': {'count': 3, 'rating': 70, 'score': 14.0}, 'MD': {'count': 1, 'rating': 35, 'score': 2.3333333333333335}, 'TD': {'count': 5, 'rating': 90, 'score': 30.0}, 'P': {'count': 2, 'rating': 55, 'score': 7.333333333333333}, 'E': {'count': 1, 'rating': 50, 'score': 3.3333333333333335}, 'F': {'count': 3, 'rating': 55, 'score': 11.0}, 'final_score': 68.0}}


## SUS and NASA-TLX Descriptive Stats

In [50]:
sus_data = []
nasa_data = []
final_scores = []

for i in range(len(sus_totals)):
    sus_data.append({
        "Participant": i+1,
        "SUS_Score": sus_totals[i]
    })

for p, fs in nasa_scores.items():
    for factor, values in fs.items():
        if factor == "final_score":
            final_scores.append({
                "Participant": p,
                "Final_Score": values
            })
        else:
            nasa_data.append({
                "Participant": p,
                "Factor": factor,
                "Count": values["count"],
                "Rating": values["rating"],
                "Score": values["score"]
            })

df1 = pd.DataFrame(sus_data)
df2 = pd.DataFrame(nasa_data)
df3 = pd.DataFrame(final_scores)
desc_sus_stats = df1["SUS_Score"].describe()
desc_nasa_stats = df2.groupby('Factor')['Score'].describe()
desc_nasa_final_stats = df3["Final_Score"].describe()

print(f"\nSUS STATS:\n\t{desc_sus_stats}")
print(f"\nNASA STATS:\n\t{desc_nasa_stats}")
print(f"\nNASA FINAL STATS:\n\t{desc_nasa_final_stats}")


SUS STATS:
	count     2.000000
mean     48.750000
std       5.303301
min      45.000000
25%      46.875000
50%      48.750000
75%      50.625000
max      52.500000
Name: SUS_Score, dtype: float64

NASA STATS:
	        count       mean        std        min        25%        50%  \
Factor                                                                 
E         2.0   6.166667   4.006938   3.333333   4.750000   6.166667   
F         2.0  12.166667   1.649916  11.000000  11.583333  12.166667   
MD        2.0   4.166667   2.592725   2.333333   3.250000   4.166667   
P         2.0   5.333333   2.828427   3.333333   4.333333   5.333333   
PD        2.0  19.000000   7.071068  14.000000  16.500000  19.000000   
TD        2.0  19.000000  15.556349   8.000000  13.500000  19.000000   

              75%        max  
Factor                        
E        7.583333   9.000000  
F       12.750000  13.333333  
MD       5.083333   6.000000  
P        6.333333   7.333333  
PD      21.500000  24.0000